In [1]:
import os
import scenic
scenic_script = "./examples/carla/car.scenic"
# scenic_script = "./examples/carla/test2.scenic"
scenario = scenic.scenarioFromFile(scenic_script)

# from scenic.domains.driving.roads import ManeuverType, Network
# map_path = '/Users/edwardkim/Desktop/Scenic-devel/examples/carla/../../tests/formats/opendrive/maps/CARLA/Town05.xodr'
# network = Network.fromFile(map_path)

/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')
/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/simulators/carla/model.scenic:56: UserWarning: the "carla" package is not installed; will not be able to run dynamic simulations
  warnings.warn('the "carla" package is not installed; '


In [2]:
from scenic.core.vectors import OrientedVector, Vector
import matplotlib.pyplot as plt
from scenic.core.type_support import toVector
from scenic.core.regions import PolygonalRegion, PointInRegionDistribution
import os.path as path
from scenic.core.regions import SectorRegion
from scenic.core.geometry import triangulatePolygon
from shapely.geometry.polygon import Polygon

obj = scenario.egoObject
label = obj.sample()
egoVisibleRegion = SectorRegion(label.position, 50, \
                                    label.heading, math.radians(130))
ego_polygon = egoVisibleRegion.polygon

intersection = []
multipolygon = scenario.egoObject.position.region

for polygon in scenario.egoObject.position.region.polygons:
    inter = ego_polygon.intersection(polygon)
    if not inter.is_empty:
        intersection.append(polygon)

for poly in intersection:
    plt.plot(*poly.exterior.xy)
plt.show()

triangles = []
for poly in intersection:
    triangles.extend(triangulatePolygon(poly))

print(len(triangles))

for t in triangles:
    plt.plot(*t.exterior.xy)
plt.plot()


NameError: name 'math' is not defined

In [3]:

from scenic.core.distributions import *
from scenic.core.vectors import Vector
from scenic.core.regions import SectorRegion
import math
import subprocess
import os


# TODO : need to handle jointly dependent objs as well
def isPositionValid(obj, label, smt_file_path, ego_info, debug=False, falseTesting=False):
    
    # TODO : need to handle jointly dependent objs as well
    if os.path.isfile(smt_file_path):
        os.remove(smt_file_path)
    
    open(smt_file_path, 'w').close()
    writeSMTtoFile(smt_file_path, '(set-logic QF_NRA)')

    # Instantiate cached_variables dictionary
    cached_variables = {}
    cached_variables['variables'] = []
    if not falseTesting:
        x = label.position.x
        y = label.position.y
    else:
        x = 23.76206016361125
        y = -201.16230930976596
        
    # Create Ego's VisibleRegion
    cached_variables['current_obj_pos'] = (x, y)
    ego_labelled_position = Vector(x,y)
    print("sampled position: "+ str(x)+" "+ str(y))
    ego_visibleDistance = ego_info['visibleDistance']
    ego_viewAngle = ego_info['viewAngle']
    ego_labelled_heading = ego_info['headingAngle']
    egoVisibleRegion = SectorRegion(ego_labelled_position, ego_visibleDistance, \
                                    ego_labelled_heading, ego_viewAngle)
    cached_variables['egoVisibleRegion'] = egoVisibleRegion
    
    # instantiate smt_var
    x_var = findVariableName(smt_file_path, cached_variables, 'x', debug=debug)
    y_var = findVariableName(smt_file_path, cached_variables, 'y', debug=debug)
    smt_var = (x_var, y_var)
    (x_cond, y_cond) = vector_operation_smt(smt_var, "equal", (str(x), str(y)))
    writeSMTtoFile(smt_file_path, smt_assert(None, x_cond))
    writeSMTtoFile(smt_file_path, smt_assert(None, y_cond))
    
    # Encode object's position to SMT formula
    obj.position.encodeToSMT(smt_file_path, cached_variables, smt_var, debug = debug)

    writeSMTtoFile(smt_file_path, "(check-sat)")
    writeSMTtoFile(smt_file_path, "(exit)")

    if not debug:
        if subprocess.call("./run_smt_encoding.sh") == 1:
            return True
        else:
            return False
    else:
        print("DEBUG MODE: NO OUTPUT WILL BE RETURNED")
            
    return None


ego_visibleDistance = 100 #meters
ego_viewAngle = 135 #deg
ego_sample = scenario.egoObject.sample()
ego_info = {
    'visibleDistance' : ego_visibleDistance,
    'viewAngle' : math.radians(ego_viewAngle), #radians,
    'headingAngle' : ego_sample.heading #radians
}

smt_file_path = './test_smt_encoding.smt2'
resetConditionedObj(scenario)
obj = scenario.egoObject
label = obj.sample()
print("label.position: ", label.position)
print(isPositionValid(obj, label, smt_file_path, ego_info, debug=False, falseTesting=False))

label.position:  (-119.24504551787531 @ 108.73702233496866)
sampled position: -119.24504551787531 108.73702233496866


TypeError: encodeToSMT() missing 1 required positional argument: 'smt_var'

In [ ]:
from scenic.core.vectors import Vector
[x1, y1] = laneSec.centerline.points[0]
[x2, y2] = laneSec.centerline.points[1]
pt1 = Vector(x1, y1)
pt2 = Vector(x2, y2)
math.degrees(pt1.angleTo(pt2))

In [ ]:
(scenario.egoObject.position)